In [1]:
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
# Read Data
data = pd.read_excel("C:/Users/Hasee/Desktop/CaseStudy/MonthlyExcessReturn.xlsx")
data = np.array(data).transpose()
print(data)

[[-9.11718567e-03  9.90486152e-03  8.43282713e-04  1.23050870e-02
   9.88632651e-03 -2.09698309e-03 -7.47316681e-03 -1.78666352e-03
  -4.69037554e-03  2.48766163e-03  1.07900661e-02  1.44686806e-03
  -5.76588778e-03  2.10004026e-03 -1.56610087e-04  1.37810051e-03
  -2.71651730e-04 -2.78681654e-03  4.05374370e-03  4.09738670e-03
  -1.05821475e-02  6.40437964e-03  9.67002405e-03  6.08175379e-03
   1.73205345e-03  5.36157435e-03  4.55463439e-03 -5.16776190e-03
  -5.25475203e-03 -3.85097918e-04 -1.36119421e-03  7.80484514e-03
  -1.35257190e-03  3.44048126e-03  2.43742263e-03 -4.31982036e-03
  -5.00823000e-03  8.81651634e-03  1.98581206e-03 -1.89254549e-03
  -3.66113800e-03 -1.09613051e-02  8.39118083e-03  1.46295783e-03
  -1.07748898e-02 -4.63636600e-03  2.40518149e-03 -1.35265698e-03
   5.31977156e-03 -5.02564036e-03 -9.28579369e-03 -1.26864616e-02
   2.52276879e-03  3.87401329e-03  5.10998141e-03 -1.41372156e-02
   5.63647067e-03  1.66597914e-04 -4.06994975e-03 -6.77646208e-03]
 [-1.4091

In [3]:
print(data.shape)

(7, 60)


In [4]:
#numpy is imported as np
#Generate Manager selection
def NextManagerSelect(ExManagerVec):
    #Number: int, between 0 and 63
    #ExManagerVec: 6 elements
    Bits = np.array([32,16,8,4,2,1])
    Number = np.sum(np.multiply(ExManagerVec,Bits))+1
    for i in range(6):
        ExManagerVec[i] = int(Number/Bits[i])#int(Number/(2**(5-i)))
        Number %= Bits[i]
    return(ExManagerVec)

In [6]:
CorrelMat = pd.read_excel("C:/Users/Hasee/Desktop/CaseStudy/CorrelationMat.xlsx")
CorrelMat = CorrelMat.to_numpy()
print(CorrelMat)
type(CorrelMat)

[[ 1.          0.11145757  0.12233711 -0.07395283  0.36937777  0.03716419
   0.04149738]
 [ 0.11145757  1.          0.0818742   0.2752012   0.09655982  0.11791328
  -0.05061536]
 [ 0.12233711  0.0818742   1.         -0.12740021  0.36027696  0.2780763
   0.13654294]
 [-0.07395283  0.2752012  -0.12740021  1.         -0.01254514  0.07725377
  -0.08824404]
 [ 0.36937777  0.09655982  0.36027696 -0.01254514  1.          0.32083468
   0.12355423]
 [ 0.03716419  0.11791328  0.2780763   0.07725377  0.32083468  1.
  -0.10802196]
 [ 0.04149738 -0.05061536  0.13654294 -0.08824404  0.12355423 -0.10802196
   1.        ]]


numpy.ndarray

In [6]:
StyleExpMat = pd.read_excel("C:/Users/Hasee/Desktop/CaseStudy/StyleExposure.xlsx")
StyleExpMat = StyleExpMat.to_numpy()
print(StyleExpMat)

[[0.35 0.2  0.1  0.2  0.45 0.15 0.2 ]
 [0.35 0.1  0.1  0.2  0.15 0.35 0.3 ]
 [0.2  0.25 0.25 0.3  0.3  0.3  0.2 ]
 [0.1  0.45 0.55 0.3  0.1  0.2  0.3 ]]


In [11]:
from scipy.optimize import minimize

# Write down given parameters
TeVec = np.array([0.02, 0.05, 0.03, 0.015, 0.03, 0.02, 0.04])
RtVec = np.array([60, 220, 180, 100, 150, 100, 180])
FeeVec = np.array([6, 32, 2, 18, 21, 16, 24])

# Initialize combination
Mg0 = np.array([1])
MgEx = np.array([0,0,0,0,0,0])

FeasibleSolutions = pd.DataFrame(columns=['W_APG','W_Gamma','W_Delta','W_Epsilon','W_Zeta','W_Kappa','W_Lambda','InformationRatio',\
                                         'Return','TrackingError','Fee','SE_Value','SE_Quality','SE_Momentum','SE_Alt/Innov'])

# The objective Function to minimize (least-squares regression)
obj = lambda WtVec,MgVec,TeVec,CorrelMat: np.dot( np.dot((MgVec*TeVec*WtVec),CorrelMat), (MgVec*TeVec*WtVec).transpose() ) \
/ (np.sum(MgVec*WtVec*RtVec)**2)

# Bounds:
bnds = [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

# Initial guess for b[1], b[2], b[3]:
xinit = np.array([0.1428, 0.1428, 0.1428, 0.1428, 0.1428, 0.1428, 0.1428])

for i in range(64):
    MgVec = np.append(Mg0,MgEx)
    # Constraint:
    cons = (
        # Weights: sum == 1
        {"type": "eq", "fun": lambda WtVec: np.sum(MgVec * WtVec)-1},
        {"type": "eq", "fun": lambda WtVec: np.sum(WtVec)-1},

        # Returns: (100 -- 120)
        {"type": "ineq", "fun": lambda WtVec: np.sum( MgVec * WtVec * RtVec ) - 100},
        #{"type": "ineq", "fun": lambda WtVec: - np.sum( MgVec * WtVec * RtVec ) + 120},
        
        # Tracking Error Square < (2.5%)**2 #2.4%
        {"type": "ineq", "fun": lambda WtVec: - np.dot(np.dot((MgVec*TeVec*WtVec),CorrelMat),(MgVec*TeVec*WtVec).transpose()) + (0.025)**2},

        # Fee < 20 bp
        {"type": "ineq", "fun": lambda WtVec: - np.sum( MgVec * WtVec * FeeVec ) + 20},
        
        # Diversification: (15% -- 35%)
        {"type": "ineq", "fun": lambda WtVec: - np.sum( MgVec * WtVec * StyleExpMat[0,:]) + 0.35},
        #{"type": "ineq", "fun": lambda WtVec:   np.sum( MgVec * WtVec * StyleExpMat[0,:]) - 0.15},
        
        {"type": "ineq", "fun": lambda WtVec: - np.sum( MgVec * WtVec * StyleExpMat[1,:]) + 0.35},
        #{"type": "ineq", "fun": lambda WtVec:   np.sum( MgVec * WtVec * StyleExpMat[1,:]) - 0.15},
        
        {"type": "ineq", "fun": lambda WtVec: - np.sum( MgVec * WtVec * StyleExpMat[2,:]) + 0.35},
        #{"type": "ineq", "fun": lambda WtVec:   np.sum( MgVec * WtVec * StyleExpMat[2,:]) - 0.15},
        
        {"type": "ineq", "fun": lambda WtVec: - np.sum( MgVec * WtVec * StyleExpMat[3,:]) + 0.35},
        #{"type": "ineq", "fun": lambda WtVec:   np.sum( MgVec * WtVec * StyleExpMat[3,:]) - 0.15},
       )
    res = minimize(obj, args=(MgVec,TeVec,CorrelMat), x0=xinit, bounds=bnds, constraints=cons)
    
    # Result: Weights, Information Ratio, Return, Tracking Error, Fee, 
    if(res.success):
        SolutionTemp = pd.Series({'W_APG':(res.x*MgVec)[0],'W_Gamma':(res.x*MgVec)[1],'W_Delta':(res.x*MgVec)[2],\
                                  'W_Epsilon':(res.x*MgVec)[3],'W_Zeta':(res.x*MgVec)[4],'W_Kappa':(res.x*MgVec)[5],\
                                  'W_Lambda':(res.x*MgVec)[6],'InformationRatio':1/sqrt(res.fun),'Return':sum(res.x*RtVec),\
                                  'TrackingError':sum(res.x*RtVec)*sqrt(res.fun),'Fee':sum(res.x*FeeVec),\
                                  'SE_Value':sum(res.x*StyleExpMat[0,:]),'SE_Quality':sum(res.x*StyleExpMat[1,:]),\
                                  'SE_Momentum':sum(res.x*StyleExpMat[2,:]),'SE_Alt/Innov':sum(res.x*StyleExpMat[3,:])})
        FeasibleSolutions = FeasibleSolutions.append(SolutionTemp,ignore_index=True)
        #print('\nManager Selection:\n',MgVec,'\nInformation Ratio:\n',1/sqrt(res.fun),'\nTracking Error:\n',
        #  sum(res.x*RtVec)*sqrt(res.fun),'\nReturn:\n',sum(res.x*RtVec),'\nWeights:\n',res.x*MgVec)
    
    MgEx = NextManagerSelect(MgEx)
    
print(FeasibleSolutions)

           W_APG   W_Gamma   W_Delta  W_Epsilon    W_Zeta   W_Kappa  W_Lambda  \
0   5.000000e-01  0.000000  0.000000   0.000000  0.000000  0.000000  0.500000   
1   7.822443e-14  0.000000  0.000000   0.000000  0.000000  1.000000  0.000000   
2   3.333333e-01  0.000000  0.000000   0.000000  0.000000  0.333333  0.333333   
3   3.333333e-01  0.000000  0.000000   0.000000  0.333333  0.000000  0.333333   
4   3.333333e-01  0.000000  0.000000   0.000000  0.333333  0.333333  0.000000   
..           ...       ...       ...        ...       ...       ...       ...   
57  1.357356e-02  0.189722  0.181318   0.202328  0.000000  0.210731  0.202328   
58  2.000000e-01  0.200000  0.200000   0.200000  0.200000  0.000000  0.000000   
59  1.357356e-02  0.197285  0.197285   0.197285  0.197285  0.000000  0.197285   
60  1.357356e-02  0.197285  0.197285   0.197285  0.197285  0.197285  0.000000   
61  1.425803e-01  0.142903  0.142903   0.142903  0.142903  0.142903  0.142903   

    InformationRatio      R

In [12]:
FeasibleSolutions.to_csv('Solutions_r2.csv')

In [166]:
'''
TeVec = np.array([0.02, 0.05, 0.03, 0.015, 0.03, 0.02, 0.04])
MgVec = np.array([1, 1, 0, 0, 0, 0, 0])
RtVec = np.array([60, 220, 180, 100, 150, 100, 180])
FeeVec = np.array([6, 32, 2, 18, 21, 16, 24])
WtVec = np.array([0.5,0.5,0,0,0,0,0])
np.dot(np.dot((MgVec*TeVec*WtVec),CorrelMat),(MgVec*TeVec*WtVec).transpose())
'''

'\nTeVec = np.array([0.02, 0.05, 0.03, 0.015, 0.03, 0.02, 0.04])\nMgVec = np.array([1, 1, 0, 0, 0, 0, 0])\nRtVec = np.array([60, 220, 180, 100, 150, 100, 180])\nFeeVec = np.array([6, 32, 2, 18, 21, 16, 24])\nWtVec = np.array([0.5,0.5,0,0,0,0,0])\nnp.dot(np.dot((MgVec*TeVec*WtVec),CorrelMat),(MgVec*TeVec*WtVec).transpose())\n'

In [1]:
(0.024)**2

0.000576